In [1]:
import numpy as np
import pandas as pd
import cv2
from keras.applications.efficientnet import EfficientNetB7
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from tqdm import tqdm

In [2]:
DATA_DIR = "../data/dog-breed-identification"

# csv 데이터 셋팅
df_train = pd.read_csv(DATA_DIR+"/labels.csv")
df_test = pd.read_csv(DATA_DIR+"/predict.csv")

# breed 파싱(중첩제거?)
targets_series = pd.Series(df_train['breed'])
# one_hot 변환(0 0 1 0)
one_hot = pd.get_dummies(targets_series, sparse = True)

# 배열로 변환
one_hot_labels = np.asarray(one_hot)

# 이미지 사이즈 정의
im_size = 90

y_train = []
x_test = []

# 예측할 분류 셋팅
i = 0
for f, breed in tqdm(df_train.values):
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1

# 예측할 데이터 셋팅
cnt = 0
for f in tqdm(df_test['id'].values):
    img = cv2.imread(DATA_DIR+"/test/{}.jpg".format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
    cnt+=1
    if(cnt == 1000):
        break

y_train_raw = np.array(y_train, np.uint8)
x_test = np.array(x_test, np.float32) / 255.


# shape 확인
print(y_train_raw.shape)
print(x_test.shape)

# 분류
num_class = y_train_raw.shape[1]

 10%|███████▍                                                                     | 999/10357 [00:01<00:16, 580.96it/s]

(10222, 120)
(1000, 90, 90, 3)


In [3]:
# 사전 교육된 기본 모델 생성
base_model = EfficientNetB7(
    # weights = 'imagenet',
    weights = None,
    include_top=False, input_shape=(im_size, im_size, 3))

# model out 정의
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 저장된 weights 로드
model.load_weights('EfficientNetB7_DOG_weights.h5')
# model.summary()

2022-09-19 03:45:57.586527: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-19 03:45:57.586630: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [4]:
import datetime
import tensorflow as tf

with tf.device('/CPU:0'):
    print("cpu predict start  :", datetime.datetime.now())
    # 예측할 데이터 결과
    preds = model.predict(x_test, verbose=1)
    print("cpu predict finish :", datetime.datetime.now())

sub = pd.DataFrame(preds)
# 열 이름을 앞서 한번의 키 인코딩으로 생성된 열 이름으로 설정
col_names = one_hot.columns.values
sub.columns = col_names

sub.head(5)

cpu predict start  : 2022-09-19 03:46:01.302181


2022-09-19 03:46:01.330902: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-19 03:46:02.260300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 20s 569ms/step
cpu predict finish : 2022-09-19 03:46:21.227225


,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0.004503,0.000098,0.019741,0.000589,0.001095,0.006962,0.000043,0.036163,0.040345,0.003970,...,0.000008,0.008938,5.586242e-21,0.000027,2.651850e-15,0.002154,1.980038e-09,0.000058,0.000062,0.007704
1,0.004273,0.000096,0.019478,0.000579,0.001070,0.006738,0.000042,0.035592,0.039180,0.003805,...,0.000007,0.008654,5.191704e-21,0.000026,2.529731e-15,0.002101,1.876589e-09,0.000056,0.000059,0.007337
2,0.005331,0.000086,0.017316,0.000522,0.001012,0.006863,0.000042,0.032093,0.035085,0.003647,...,0.000007,0.007944,3.587259e-21,0.000024,2.066930e-15,0.002022,1.521264e-09,0.000047,0.000054,0.006822
3,0.004479,0.000098,0.019712,0.000579,0.001113,0.006957,0.000044,0.036827,0.041055,0.004040,...,0.000008,0.009133,5.468201e-21,0.000027,2.579665e-15,0.002165,1.960731e-09,0.000058,0.000061,0.007690
4,0.004369,0.000099,0.019773,0.000591,0.001091,0.006711,0.000044,0.035888,0.039040,0.003932,...,0.000008,0.008772,5.852885e-21,0.000027,2.713797e-15,0.002109,1.985644e-09,0.000057,0.000060,0.007628


In [5]:
import datetime

print("gpu predict start  :", datetime.datetime.now())
# 예측할 데이터 결과
preds = model.predict(x_test, verbose=1)
print("gpu predict finish :", datetime.datetime.now())

sub = pd.DataFrame(preds)
# 열 이름을 앞서 한번의 키 인코딩으로 생성된 열 이름으로 설정
col_names = one_hot.columns.values
sub.columns = col_names

sub.head(5)

gpu predict start  : 2022-09-19 03:46:21.255632


2022-09-19 03:46:21.415345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 9s 202ms/step
gpu predict finish : 2022-09-19 03:46:30.371222


,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0.004504,0.000098,0.019741,0.000589,0.001095,0.006963,0.000043,0.036167,0.040349,0.003971,...,0.000008,0.008938,5.587663e-21,0.000027,2.652132e-15,0.002155,1.980353e-09,0.000058,0.000062,0.007706
1,0.004273,0.000096,0.019479,0.000579,0.001070,0.006738,0.000042,0.035591,0.039183,0.003806,...,0.000007,0.008655,5.190773e-21,0.000026,2.529338e-15,0.002102,1.876655e-09,0.000056,0.000059,0.007339
2,0.005332,0.000086,0.017317,0.000522,0.001012,0.006862,0.000042,0.032085,0.035095,0.003646,...,0.000007,0.007947,3.587646e-21,0.000024,2.067105e-15,0.002022,1.520938e-09,0.000047,0.000054,0.006820
3,0.004479,0.000098,0.019712,0.000580,0.001113,0.006957,0.000044,0.036847,0.041057,0.004042,...,0.000008,0.009133,5.469254e-21,0.000027,2.579597e-15,0.002165,1.961021e-09,0.000058,0.000061,0.007690
4,0.004369,0.000099,0.019769,0.000590,0.001091,0.006712,0.000044,0.035886,0.039038,0.003932,...,0.000008,0.008772,5.854330e-21,0.000027,2.713415e-15,0.002109,1.986090e-09,0.000057,0.000060,0.007628
